In [1]:
# set the seed value for reproducible results
from numpy.random import seed
seed(11)

In [2]:
# import dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow
tensorflow.keras.__version__

C:\Users\admin\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\anaconda3\envs\PythonData\lib\site-packages

'2.2.4-tf'

In [4]:
# bring in the data
df = pd.read_csv('../Data/threecleaneddata.csv')
df.head()

,Unnamed: 0,url,Title_Word_Count,Content_Word_Count,Unique_Word_Rate,Non_Stop_Word_Rate,Unique_Non_Stop_Word_Rate,Number_of_Links,Number_of_Mashable_Links,Images,...,Maximum_Positive_Polarity,Average_Negative_Polarity,Minimum_Negative_Polarity,Maximum_Negative_Polarity,Title_Subjectivity,Title_Polarity,Absolute_Subjectivity_Level,Absolute_Polarity_Level,Shares,Category
0,0,http://mashable.com/2013/01/07/amazon-instant-...,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,1,http://mashable.com/2013/01/07/ap-samsung-spon...,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,2,http://mashable.com/2013/01/07/apple-40-billio...,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,1
3,3,http://mashable.com/2013/01/07/astronaut-notre...,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,1
4,4,http://mashable.com/2013/01/07/att-u-verse-apps/,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0


## Data Pre-Processing

In [5]:
X = df.drop(['Unnamed: 0', 'url', 'Category', 'Shares'], axis=1)

y = df['Category']
print(X.shape, y.shape)

(39644, 58) (39644,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_test_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

## Create a Deep Learning Model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=58))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [16]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               5900      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 16,303
Trainable params: 16,303
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
29733/29733 - 2s - loss: 1.0302 - acc: 0.4772
Epoch 2/60
29733/29733 - 2s - loss: 0.9927 - acc: 0.5055
Epoch 3/60
29733/29733 - 2s - loss: 0.9818 - acc: 0.5105
Epoch 4/60
29733/29733 - 2s - loss: 0.9722 - acc: 0.5205
Epoch 5/60
29733/29733 - 2s - loss: 0.9635 - acc: 0.5254
Epoch 6/60
29733/29733 - 2s - loss: 0.9551 - acc: 0.5366
Epoch 7/60
29733/29733 - 2s - loss: 0.9453 - acc: 0.5435
Epoch 8/60
29733/29733 - 2s - loss: 0.9331 - acc: 0.5519
Epoch 9/60
29733/29733 - 2s - loss: 0.9238 - acc: 0.5578
Epoch 10/60
29733/29733 - 2s - loss: 0.9140 - acc: 0.5656
Epoch 11/60
29733/29733 - 1s - loss: 0.9026 - acc: 0.5732
Epoch 12/60
29733/29733 - 1s - loss: 0.8900 - acc: 0.5798
Epoch 13/60
29733/29733 - 1s - loss: 0.8816 - acc: 0.5888
Epoch 14/60
29733/29733 - 1s - loss: 0.8693 - acc: 0.5951
Epoch 15/60
29733/29733 - 1s - loss: 0.8599 - acc: 0.5991
Epoch 16/60
29733/29733 - 1s - loss: 0.8496 - acc: 0.6066
Epoch 17/60
29733/29733 - 1s - loss: 0.8389 - acc: 0.6137
Epoch 18/60
29733/29733

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

9911/9911 - 0s - loss: 1.7058 - acc: 0.4328
Normal Neural Network - Loss: 1.7057724897272832, Accuracy: 0.43275147676467896
